In [2]:
from selenium import webdriver


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv, time, random

def scrape_website(link, year_query="193"):
    driver = webdriver.Chrome()
    wait   = WebDriverWait(driver, 10)         
    driver.get(link)

    wait.until(EC.element_to_be_clickable(
        (By.XPATH, '//*[@id="root"]/div[1]/main/div/div[3]/button'))).click()

    # 2️⃣ type the query and submit
    search = driver.find_element(
        By.CSS_SELECTOR, "input[placeholder='Type words from song title']")
    search.clear()
    search.send_keys(f"year:{year_query}")

    driver.find_element(
        By.XPATH, '//*[@id="root"]/div[1]/main/div/div[2]/div/form/button[2]'
    ).click()

    wait.until(EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "table tbody tr")))

    rows   = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    songs  = []

    for r in rows:
        cells = r.find_elements(By.TAG_NAME, "td")
        if len(cells) < 6:                        
            continue
        songs.append({
            "title"    : cells[0].text.strip(),
            "album"    : cells[1].text.strip(),
            "year"     : cells[2].text.strip(),
            "composer" : cells[3].text.strip(),
            "lyricist" : cells[4].text.strip(),
            "singer"   : cells[5].text.strip(),
        })

    with open(f"songs_{year_query}.csv", "w", newline='', encoding="utf‑8") as f:
        writer = csv.DictWriter(f, fieldnames=songs[0].keys())
        writer.writeheader()
        writer.writerows(songs)

    time.sleep(random.uniform(0.5, 1.0))
    driver.quit()



# demo run
scrape_website("https://new.giitaayan.com/", year_query="193")
